In [ ]:
!pip install scikit-optimize
!pip install -U pymoo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
from skopt import gp_minimize
import matplotlib.pyplot as plt

In [ ]:
# Carregar os dados
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalizar os dados
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

In [ ]:
def build_cnn_model():
    # Criar o modelo da rede neural convolucional
    model = keras.Sequential([
        layers.Input(shape=(28, 28, 1)),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation="relu"),
        layers.Dense(10, activation="softmax")
    ])

    return model

In [ ]:
def chebyshev_loss(y_true, y_pred, model, lambda1, lambda2):
    # Função de perda Categorical Crossentropy
    ce_loss = keras.losses.CategoricalCrossentropy()(y_true, y_pred)

    # Função de regularização L2
    l2_loss = sum([tf.nn.l2_loss(layer.kernel) for layer in model.layers if 'Conv2D' in str(layer) or 'Dense' in str(layer)])

    # Combinar as funções de perda com os pesos de Chebyshev
    combined_loss = lambda1 * ce_loss + lambda2 * l2_loss

    return combined_loss

In [ ]:
def objective(params):
    lambda1, lambda2 = params
    
    # Construir o modelo da CNN
    model = build_cnn_model()

    # Compilar o modelo com a função de perda personalizada e otimizador
    model.compile(loss=lambda y_true, y_pred: chebyshev_loss(y_true, y_pred, model, lambda1, lambda2), optimizer=keras.optimizers.Adam(), metrics=["accuracy"])

    # Treinar o modelo
    model.fit(x_train, y_train, batch_size=128, epochs=2, validation_split=0.1, verbose=1)

    # Avaliar o desempenho do modelo
    score = model.evaluate(x_test, y_test, verbose=0)
    
    return score[0]

In [ ]:
space = [(0.0001, 0.01, "uniform"), (0.0001, 0.01, "uniform")]

result = gp_minimize(func=objective, dimensions=space, n_calls=10, random_state=42, verbose=1)

#Imprimir os melhores hiperparâmetros encontrados
best_lambda1, best_lambda2 = result.x
print(f"Melhores hiperparâmetros encontrados: lambda1 = {best_lambda1}, lambda2 = {best_lambda2}")

Iteration No: 1 started. Evaluating function at random point.
Epoch 1/2
422/422 [==============================] - 63s 146ms/step - loss: 0.0302 - accuracy: 0.6877 - val_loss: 0.0151 - val_accuracy: 0.8095
Epoch 2/2
422/422 [==============================] - 47s 111ms/step - loss: 0.0155 - accuracy: 0.7741 - val_loss: 0.0152 - val_accuracy: 0.7815
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 146.3534
Function value obtained: 0.0154
Current minimum: 0.0154
Iteration No: 2 started. Evaluating function at random point.
Epoch 1/2
422/422 [==============================] - 50s 114ms/step - loss: 0.0619 - accuracy: 0.1221 - val_loss: 0.0180 - val_accuracy: 0.1050
Epoch 2/2
422/422 [==============================] - 49s 116ms/step - loss: 0.0180 - accuracy: 0.1132 - val_loss: 0.0180 - val_accuracy: 0.1050
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 104.4202
Function value obtained: 0.0180
Current minimum: 0.0154
Iteration No: 3 started. Evaluatin

In [ ]:
#Construir e treinar o melhor base
base_model = build_cnn_model()
base_model.compile(loss="categorical_crossentropy", optimizer=keras.optimizers.Adam(), metrics=["accuracy"])
base_model.fit(x_train, y_train, batch_size=128, epochs=2, validation_split=0.1, verbose=1)

#Avaliar o melhor base
score = base_model.evaluate(x_test, y_test, verbose=1)
print(f"Test loss: {score[0]}, Test accuracy: {score[1]}")

Epoch 1/2
422/422 [==============================] - 62s 145ms/step - loss: 0.2213 - accuracy: 0.9383 - val_loss: 0.0606 - val_accuracy: 0.9838
Epoch 2/2
313/313 [==============================] - 4s 12ms/step - loss: 0.0398 - accuracy: 0.9875
Test loss: 0.03984352946281433, Test accuracy: 0.987500011920929


In [ ]:
#Construir e treinar o melhor modelo
best_model = build_cnn_model()
best_model.compile(loss=lambda y_true, y_pred: chebyshev_loss(y_true, y_pred, best_model, best_lambda1, best_lambda2),
optimizer=keras.optimizers.Adam(), metrics=["accuracy"])
best_model.fit(x_train, y_train, batch_size=128, epochs=2, validation_split=0.1, verbose=1)

#Avaliar o melhor modelo
score = best_model.evaluate(x_test, y_test, verbose=1)
print(f"Test loss: {score[0]}, Test accuracy: {score[1]}")